In [2]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Dataset
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig
from azureml.core import Model

from azureml.exceptions import ComputeTargetException

# own modules
import sys
sys.path.append('../')
from config.config_training import script_params_remote

In [3]:
# important parameters
compute_name = 'dp-100-v2'
project_folder = '../'
remote = False

# 1. Configure Workspace

In [4]:
ws = Workspace.from_config('../config/config.json')

# 2. Configure Compute Target

In [9]:
if remote:
    try:
        comput_target = ComputeTarget(workspace=ws, name= compute_name)
        print('Comput target exists!')
    except:
        config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_VS2'
                                                              ,vm_priority='lowpriority'
                                                              ,min_nodes=0
                                                              , max_nodes=1)
        compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
        compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
else:
    compute_target = 'local'

# 3. Configurate Dependencies

In [6]:
if remote:
    env = Environment.from_conda_specification(name = 'sklearn-env', file_path = '../config/conda_dependencies.yml')
else:
    env = Environment('user-managed-env')
    env.python.user_managed_dependencies = True

# 4. Get Data

Run this in conda shell before starting jupyter lab: __export LC_ALL=en_US.UTF-8__

In [7]:
tab_ds = Dataset.get_by_name(ws, 'diabetes dataset')
script_params_remote.extend(['--ds', tab_ds.as_named_input('diabetes_dataset')])

# 5. Configure Experiment

In [16]:
src = ScriptRunConfig(source_directory=project_folder
                     ,script='./src/train.py'
                     ,compute_target=compute_target
                     ,arguments=script_params_remote
                     ,environment=env)

In [17]:
experiment = Experiment(workspace=ws, name='test_v5')

# 6. Run Experiment und Monitor Training

In [18]:
run = experiment.submit(src)
run.wait_for_completion(show_output=True)

RunId: test_v5_1612217479_193aacfb
Web View: https://ml.azure.com/experiments/test_v5/runs/test_v5_1612217479_193aacfb?wsid=/subscriptions/c2cbd114-566e-42ab-a9f3-ae8000df45a0/resourcegroups/certification_dp100/workspaces/dp_100_v2

Streaming azureml-logs/60_control_log.txt

[2021-02-01T22:11:25.037025] Entering context manager injector.
[2021-02-01T22:11:27.023481] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 5171
Running: ['/bin/bash', '/private/var/folders/2x/5phlmdsd0td5pb7srj3fpqth0000gn/T/azureml_runs/test_v5_1612217479_193aacfb/azureml-environment-setup/conda_env_checker.sh']


Running: ['python', 'azureml-setup/run_script.py', 'python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'RunHistory:context_managers.RunHistory', '-i', 'TrackUserError:context_managers.TrackUserError', 'src/tr

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Local execution of User Script failed. Details can be found in azureml-logs/60_control_log.txt log file.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Local execution of User Script failed. Details can be found in azureml-logs/60_control_log.txt log file.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

# 7. Add trained motel to workspace

In [12]:
model = Model.register(workspace=ws
                       ,model_path='../trained_models/diabetes_model.pkl'
                       ,model_name='Diabetes_Classifier'
                       ,description='First trained model with Azure ML'
                       ,tags={'data-format': 'csv'}
                       ,properties={'Accuracy': run.get_metrics()['Accuracy']}
                       ,model_framework=Model.Framework.SCIKITLEARN  # Framework used to create the model.
                       ,model_framework_version='0.23.2'
                       ,datasets=[('training_data', tab_ds)]
                       )

KeyError: 'Accuracy'